# Parte II: Preprocesamiento y Optimización

In [5]:
#importamos todas las librerias necesarias
import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import make_classification
from sklearn.base import BaseEstimator, TransformerMixin

In [66]:
# cargamos el dataset
ruta='../selected_dataset/selected_dataset.csv'
df = pd.read_csv(filepath_or_buffer=ruta, sep=',')
df.shape # mostramos las dimensiones de filas y columnas

(569, 10)

## Parte 1: Preprocesamiento de Datos

### 1. Limpieza de Datos:
* Tratar los valores nulos utilizando técnicas adecuadas (imputación, eliminación, etc.).
* Manejar los outliers mediante técnicas de filtrado o transformación

In [7]:
df.isnull().sum() # verificamos la existencia de valores nulos

id              0
Name            0
Writer          1
Likes           0
Genre           0
Rating          0
Subscribers     0
Summary         0
Update          0
Reading Link    0
dtype: int64

In [8]:
df.info()  # obtenemos informacin del dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            569 non-null    int64  
 1   Name          569 non-null    object 
 2   Writer        568 non-null    object 
 3   Likes         569 non-null    object 
 4   Genre         569 non-null    object 
 5   Rating        569 non-null    float64
 6   Subscribers   569 non-null    object 
 7   Summary       569 non-null    object 
 8   Update        569 non-null    object 
 9   Reading Link  569 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 44.6+ KB


In [9]:
df.head() # mostramos las primeras 5 lineas

,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link
0,0,Let's Play,Leeanne M. Krecic (Mongie),30.6M,Romance,9.62,4.2M,"She's young, single and about to achieve her d...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/l...
1,1,True Beauty,Yaongyi,39.9M,Romance,9.60,6.4M,"After binge-watching beauty videos online, a s...",UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/truebeauty...
2,2,Midnight Poppy Land,Lilydusk,10.4M,Romance,9.81,2.1M,After making a grisly discovery in the country...,UP EVERY SATURDAY,https://www.webtoons.com/en/romance/midnight-p...
3,3,Age Matters,Enjelicious,25.9M,Romance,9.79,3.5M,She's a hopeless romantic who's turning 30's ...,UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/age-matter...
4,4,Unholy Blood,Lina Im / Jeonghyeon Kim,9.9M,Supernatural,9.85,1.5M,When vampires destroy her chance to have the n...,UP EVERY THURSDAY,https://www.webtoons.com/en/supernatural/unhol...


In [69]:
# Función para convertir valores con 'M', 'K' a números
def convert_to_numeric(value):
    if isinstance(value, str):
        value = value.replace(',', '')  # Eliminar comas
        value = value.upper()  # Convertimos a mayuscula para que pueda encontrar todas las coincidencias
        if 'M' in value:
            return float(value.replace('M', '')) * 1e6  # Convertir 'M' a millones
        elif 'K' in value:
            return float(value.replace('K', '')) * 1e3  # Convertir 'K' a miles
    return float(value) # rotorna el valor nuevo

In [70]:
# aplicamos la conversión a las columnas 'Likes' y 'Subscribers'
df['Likes'] = df['Likes'].apply(convert_to_numeric)  # aplicamos la función convert_to_numeric a cada valor de la columna Likes
df['Subscribers'] = df['Subscribers'].apply(convert_to_numeric)  # aplicamos la función a cada valor de la columna Subscribers

# verificamos los primeros registros para asegurarnos que la conversión fue exitosa
print(df[['Likes', 'Subscribers']].head()) 


        Likes  Subscribers
0  30600000.0    4200000.0
1  39900000.0    6400000.0
2  10400000.0    2100000.0
3  25900000.0    3500000.0
4   9900000.0    1500000.0


In [71]:
# seleccionamos las columnas numéricas a verificar
columns_to_check = ['Likes', 'Subscribers', 'Rating']  # especificamos las columnas que contienen valores numéricos para verificar outliers

# calculamos Q1 (primer cuartil) y Q3 (tercer cuartil) para cada columna
Q1 = df[columns_to_check].quantile(0.25)  # calculamos el primer cuartil (25%) para cada columna seleccionada
Q3 = df[columns_to_check].quantile(0.75)  # calculamos el tercer cuartil (75%) para cada columna seleccionada
# calculamos el IQR (Rango Intercuartílico)
IQR = Q3 - Q1  # calculamos la diferencia entre el tercer y el primer cuartil, que es el IQR

# establecemos los límites para detectar outliers
lower_bound = Q1 - 1.5 * IQR  # calculamos el límite inferior para detectar valores atípicos (outliers)
upper_bound = Q3 + 1.5 * IQR  # calculamos el límite superior para detectar valores atípicos (outliers)

# filtramos los outliers (dejamos solo los valores dentro del rango)
df_filtered = df[~((df[columns_to_check] < lower_bound) | (df[columns_to_check] > upper_bound)).any(axis=1)]  # filtramos el DataFrame eliminando filas con valores fuera de los límites

# imprimimos el tamaño del DataFrame original y el filtrado
print(f'Tamaño original del DataFrame: {df.shape[0]}')  # imprimimos el número de filas del DataFrame original
print(f'Tamaño después de eliminar outliers: {df_filtered.shape[0]}')  # imprimimos el número de filas después de eliminar los outliers


Tamaño original del DataFrame: 569
Tamaño después de eliminar outliers: 458


### 2. Transformación de Columnas:
* Utilizar ColumnTransformer para aplicar transformaciones específicas a diferentes columnas.
* Realizar codificación de variables categóricas utilizando técnicas como One-Hot Encoding.
* Escalar las variables numéricas usando StandardScaler u otros métodos de normalización.

In [72]:
# definimos el ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        # categóricas: 'Genre' y 'Writer' codificadas con OneHotEncoder
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Genre', 'Writer']),
        
        # numéricas: 'Likes', 'Subscribers', 'Rating' escaladas con StandardScaler
        ('num', StandardScaler(), ['Likes', 'Subscribers', 'Rating'])
    ])

### 3. Creación de Pipelines:
* Crear pipelines utilizando Pipeline de sklearn para automatizar el preprocesamiento de datos y asegurar la reproducibilidad.
* Incluir todos los pasos de preprocesamiento en el pipeline.

In [73]:
# creamos el pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])  # definimos un pipeline que incluye el paso de preprocesamiento

# aplicamos la transformación
df_transformed = pipeline.fit_transform(df)  # ajustamos y transformamos los datos de 'df' utilizando el pipeline

# verificamos el número de columnas generadas
cat_columns = pipeline.named_steps['preprocessor'].transformers_[0][1].get_feature_names_out(['Genre', 'Writer'])  # obtenemos los nombres de las columnas generadas por el codificador OneHotEncoder para las columnas 'Genre' y 'Writer'
print(f"Número de columnas generadas: {len(cat_columns)}")  # mostramos la cantidad de columnas generadas por la codificación de 'Genre' y 'Writer'

Número de columnas generadas: 506


In [74]:
# unimos las columnas transformadas con las columnas no transformadas (Likes, Rating, Subscribers)
columns = list(cat_columns) + ['Likes', 'Subscribers', 'Rating']
# verificamos si la matriz transformada es dispersa (sparse matrix)
if hasattr(df_transformed, 'toarray'):
    df_transformed = df_transformed.toarray()  # convertimos la matriz dispersa a densa (array numpy)
# convertimos la matriz transformada a DataFrame
df_transformed = pd.DataFrame(df_transformed, columns=columns)
# mostramos el DataFrame transformado
print(df_transformed.head())

   Genre_Action  Genre_Comedy  Genre_Drama  Genre_Fantasy  Genre_Heartwarming  \
0           0.0           0.0          0.0            0.0                 0.0   
1           0.0           0.0          0.0            0.0                 0.0   
2           0.0           0.0          0.0            0.0                 0.0   
3           0.0           0.0          0.0            0.0                 0.0   
4           0.0           0.0          0.0            0.0                 0.0   

   Genre_Historical  Genre_Horror  Genre_Informative  Genre_Mystery  \
0               0.0           0.0                0.0            0.0   
1               0.0           0.0                0.0            0.0   
2               0.0           0.0                0.0            0.0   
3               0.0           0.0                0.0            0.0   
4               0.0           0.0                0.0            0.0   

   Genre_Romance  ...  Writer_walkingnorth  Writer_waroo  Writer_yee seon  \
0        

## Parte 2: Selección de Técnica de Machine Learning

### 1. Entrenamiento Inicial:
* Entrenar múltiples modelos de machine learning (por ejemplo, Regresión Lineal, KNN, Árbol de Decisión, Random Forest, XGBoost, LGBM).
* Evaluar los modelos utilizando validación cruzada y seleccionar el modelo con el mejor rendimiento inicial.

In [75]:
df.head() # mostramos las 5 primeras filas 

,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link
0,0,Let's Play,Leeanne M. Krecic (Mongie),30600000.0,Romance,9.62,4200000.0,"She's young, single and about to achieve her d...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/l...
1,1,True Beauty,Yaongyi,39900000.0,Romance,9.60,6400000.0,"After binge-watching beauty videos online, a s...",UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/truebeauty...
2,2,Midnight Poppy Land,Lilydusk,10400000.0,Romance,9.81,2100000.0,After making a grisly discovery in the country...,UP EVERY SATURDAY,https://www.webtoons.com/en/romance/midnight-p...
3,3,Age Matters,Enjelicious,25900000.0,Romance,9.79,3500000.0,She's a hopeless romantic who's turning 30's ...,UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/age-matter...
4,4,Unholy Blood,Lina Im / Jeonghyeon Kim,9900000.0,Supernatural,9.85,1500000.0,When vampires destroy her chance to have the n...,UP EVERY THURSDAY,https://www.webtoons.com/en/supernatural/unhol...


In [76]:
class CurrencyToNumeric(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # no se necesita ajustar nada

    def transform(self, X):
        # realizamos la conversión, eliminando signos de moneda y comas
        return X.replace({'\$': '', ',': ''}, regex=True).astype(float) # convertimos los valores a tipo float después de eliminar los símbolos

In [77]:
# crear columna binaria 'high_rating' (1 si Rating > mediana, sino 0)
median_rating = df['Rating'].median()  # calculamos la mediana de la columna 'Rating'
df['high_rating'] = (df['Rating'] > median_rating).astype(int)  # asignamos 1 si Rating es mayor que la mediana, sino 0

# selección de características (X) y variable objetivo (y)
X = df[['Genre', 'Writer', 'Likes', 'Subscribers', 'Rating']]  # seleccionamos las características para el modelo
y = df['high_rating']  # definimos 'high_rating' como la variable objetivo

# definir el pipeline con las transformaciones y modelos
pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('genre_writer', OneHotEncoder(handle_unknown='ignore'), ['Genre', 'Writer']),  # realizamos codificación One-Hot para las columnas 'Genre' y 'Writer'
            ('currency', CurrencyToNumeric(), ['Likes', 'Subscribers']),  # convertimos 'Likes' y 'Subscribers' a formato numérico
            ('scaler', StandardScaler(), ['Rating'])  # escalamos la columna 'Rating'
        ],
        remainder='passthrough'  # mantenemos las demás columnas sin cambios
    )),
    ('classifier', LogisticRegression())  # definimos el clasificador (Regresión Logística en este caso)
])

In [78]:
# dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)  # separamos el conjunto de datos en entrenamiento (70%) y prueba (30%)
# ajustamos el pipeline a los datos de entrenamiento
pipeline.fit(X_train, y_train)  # entrenamos el modelo utilizando los datos de entrenamiento (X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('genre_writer',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Genre', 'Writer']),
                                                 ('currency',
                                                  CurrencyToNumeric(),
                                                  ['Likes', 'Subscribers']),
                                                 ('scaler', StandardScaler(),
                                                  ['Rating'])])),
                ('classifier', LogisticRegression())])

In [79]:
# evaluamos el modelo en el conjunto de prueba
print("Regresión Logística - Accuracy en Test: ", pipeline.score(X_test, y_test))

Regresión Logística - Accuracy en Test:  0.9473684210526315


In [81]:
# cambiamos el modelo del pipeline a KNeighborsClassifier con 3 vecinos
pipeline.set_params(classifier=KNeighborsClassifier(n_neighbors=3))  # sustituimos el clasificador actual por KNeighborsClassifier con 3 vecinos
# ajustamos el pipeline con el nuevo clasificador a los datos de entrenamiento
pipeline.fit(X_train, y_train)  # entrenamos el modelo con los datos de entrenamiento utilizando el nuevo clasificador


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('genre_writer',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Genre', 'Writer']),
                                                 ('currency',
                                                  CurrencyToNumeric(),
                                                  ['Likes', 'Subscribers']),
                                                 ('scaler', StandardScaler(),
                                                  ['Rating'])])),
                ('classifier', KNeighborsClassifier(n_neighbors=3))])

In [82]:
# evaluamos el modelo con K-Nearest Neighbors en el conjunto de prueba
print("K-Nearest Neighbors - Accuracy en Test: ", pipeline.score(X_test, y_test))

K-Nearest Neighbors - Accuracy en Test:  0.6608187134502924


In [83]:
# cambiamos el modelo del pipeline a DecisionTreeClassifier (Árbol de Decisión) con semilla aleatoria 42
pipeline.set_params(classifier=DecisionTreeClassifier(random_state=42))
# ajustamos el pipeline con el nuevo clasificador a los datos de entrenamiento
pipeline.fit(X_train, y_train)  # entrenamos el modelo con los datos de entrenamiento utilizando el árbol de decisión


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('genre_writer',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Genre', 'Writer']),
                                                 ('currency',
                                                  CurrencyToNumeric(),
                                                  ['Likes', 'Subscribers']),
                                                 ('scaler', StandardScaler(),
                                                  ['Rating'])])),
                ('classifier', DecisionTreeClassifier(random_state=42))])

In [84]:
# evaluamos el modelo con árbol de decisión en el conjunto de prueba
print("Árbol de Decisión - Accuracy en Test: ", pipeline.score(X_test, y_test))  # calculamos y mostramos la precisión del modelo en los datos de prueba con el árbol de decisión

Árbol de Decisión - Accuracy en Test:  1.0


### 2. Comparación de Modelos:
* Comparar los modelos utilizando métricas de rendimiento relevantes (exactitud, precisión, recall, F1-Score, ROC-AUC, etc.).
* Seleccionar la técnica de machine learning más adecuada basándose en las métricas y la naturaleza del problema.

In [92]:
# creamos un diccionario de los modelos a comparar
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=3),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}
# generamos los resultados y lo almacenamos en results
results = {}
for model_name, model in models.items():
    # Crear y ajustar el pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('genre_writer', OneHotEncoder(handle_unknown='ignore'), ['Genre', 'Writer']),
                ('currency', CurrencyToNumeric(), ['Likes', 'Subscribers']),
                ('scaler', StandardScaler(), ['Rating'])
            ],
            remainder='passthrough'
        )),
        ('classifier', model)
    ])
    
    # entrenamos el modelo
    pipeline.fit(X_train, y_train)
    
    # realizamos la prediccion sobre el conjunto de prueba
    y_pred = pipeline.predict(X_test)
    
    # calculamos las métricas de rendimiento
    accuracy = accuracy_score(y_test, y_pred) # precisión
    precision = precision_score(y_test, y_pred) # precisión
    recall = recall_score(y_test, y_pred) # sensibilidad
    f1 = f1_score(y_test, y_pred) # puntuación F1
    auc_roc = roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]) # área bajo la curva ROC
    
    # guardamos los resultados
    results[model_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': auc_roc
    }
    
    # mostramos las métricas de cada modelo
    print("\n" + "="*50 )
    print(f"Resultados para {model_name}:")
    print("-"*50 )
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"ROC-AUC: {auc_roc:.4f}")
    


Resultados para Logistic Regression:
--------------------------------------------------
Accuracy: 0.9474
Precision: 0.9481
Recall: 0.9359
F1-Score: 0.9419
ROC-AUC: 0.9876

Resultados para K-Nearest Neighbors:
--------------------------------------------------
Accuracy: 0.6608
Precision: 0.6087
Recall: 0.7179
F1-Score: 0.6588
ROC-AUC: 0.7153

Resultados para Decision Tree:
--------------------------------------------------
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
ROC-AUC: 1.0000


In [93]:
# mostramos resumen de resultados
results_df = pd.DataFrame(results).T
print("Comparación de Modelos:")
print(results_df)

Comparación de Modelos:
                     Accuracy  Precision    Recall  F1-Score   ROC-AUC
Logistic Regression  0.947368   0.948052  0.935897  0.941935  0.987593
K-Nearest Neighbors  0.660819   0.608696  0.717949  0.658824  0.715329
Decision Tree        1.000000   1.000000  1.000000  1.000000  1.000000


## Parte 3: Optimización de Hiperparámetros

### 1. GridSearchCV:
* Implementar GridSearchCV para realizar una búsqueda exhaustiva de los mejores hiperparámetros para el modelo seleccionado.
* Definir el espacio de búsqueda para los hiperparámetros relevantes.

In [94]:
# definimos el espacio de búsqueda de hiperparámetros por modelo
param_grid = [
    # hiperparámetros para Logistic Regression
    {
        'classifier': [LogisticRegression()],
        'classifier__C': [0.1, 1, 10],  # controla la regularización
        'classifier__penalty': ['l2', 'none'],  # tipo de regularización (l2 es común)
        'classifier__solver': ['liblinear', 'newton-cg'],  # solvers disponibles
        'classifier__max_iter': [100, 200],  # numero de iteraciones
    },
    
    # hiperparámetros para K-Nearest Neighbors (KNN)
    {
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': [3, 5, 7, 9],  # número de vecinos
        'classifier__weights': ['uniform', 'distance'],  # ponderación de los vecinos
        'classifier__metric': ['euclidean', 'manhattan'],  # métrica de distancia
    },
    
    # hiperparámetros para Decision Tree Classifier
    {
        'classifier': [DecisionTreeClassifier()],
        'classifier__max_depth': [3, 5, 7, None],  # profundidad máxima del árbol (None no limita la profundidad)
        'classifier__min_samples_split': [2, 5, 10],  # mínimas muestras para dividir un nodo
        'classifier__min_samples_leaf': [1, 2, 4],  # mínimas muestras para ser hoja
        'classifier__criterion': ['gini', 'entropy'],  # criterio de división del árbol
        'classifier__max_features': ['auto', 'sqrt', 'log2'],  # características máximas a considerar en cada división
    }
]

# configuramos GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='accuracy', verbose=1)

# ejecutamos la búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 256 candidates, totalling 1280 fits


/home/cvelazquez/.local/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
/home/cvelazquez/anaconda3/envs/ds_mitic/lib/python3.10/site-packages/sklearn/utils/optimize.py:99: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(
/home/cvelazquez/.local/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
/home/cvelazquez/anaconda3/envs/ds_mitic/lib/python3.10/site-packages/sklearn/utils/optimize.py:99: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(
/home/cvelazquez/anaconda3/envs/ds_mitic/lib/python3.10/site-packages/sklearn/utils/optimize.py:318: ConvergenceWarning: newton-cg failed to converge at loss = 0.40949698654973066.

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('genre_writer',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Genre',
                                                                          'Writer']),
                                                                        ('currency',
                                                                         CurrencyToNumeric(),
                                                                         ['Likes',
                                                                          'Subscribers']),
                                                                        ('scaler',
                                                                         StandardScaler(),
                                                                         ['Rating'])])),
                                       ('classifier',
                                        DecisionTreeClassifier(random_state=42))]),
             n_j...
                          'classifier__n_neighbors': [3, 5, 7, 9],
                          'classifier__weights': ['uniform', 'distance']},
                         {'classifier': [DecisionTreeClassifier()],
                          'classifier__criterion': ['gini', 'entropy'],
                          'classifier__max_depth': [3, 5, 7, None],
                          'classifier__max_features': ['auto', 'sqrt', 'log2'],
                          'classifier__min_samples_leaf': [1, 2, 4],
                          'classifier__min_samples_split': [2, 5, 10]}],
             scoring='accuracy', verbose=1)

In [95]:
# mostramos los mejores parámetros y el mejor modelo
print("Mejores parámetros encontrados:", grid_search.best_params_)
print("Mejor modelo encontrado:", grid_search.best_estimator_)

# evaluamos el modelo con el mejor conjunto de parámetros
y_pred = grid_search.predict(X_test)

# calcular métricas de rendimiento
accuracy = accuracy_score(y_test, y_pred) # precision
precision = precision_score(y_test, y_pred) # precision
recall = recall_score(y_test, y_pred) # sensibilidad
f1 = f1_score(y_test, y_pred) # puntuacion F1
auc_roc = roc_auc_score(y_test, grid_search.predict_proba(X_test)[:, 1]) # area bajo la curva 

print("\n")
print("="*50)
print(f"Métricas para el mejor modelo:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {auc_roc:.4f}")

Mejores parámetros encontrados: {'classifier': DecisionTreeClassifier(), 'classifier__criterion': 'entropy', 'classifier__max_depth': None, 'classifier__max_features': 'log2', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 10}
Mejor modelo encontrado: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('genre_writer',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Genre', 'Writer']),
                                                 ('currency',
                                                  CurrencyToNumeric(),
                                                  ['Likes', 'Subscribers']),
                                                 ('scaler', StandardScaler(),
                                                  ['Rating'])])),
                ('classifier',
                 Deci

### 2. RandomizedSearchCV:
* Implementar RandomizedSearchCV para realizar una búsqueda aleatoria de los mejores hiperparámetros, especialmente útil si el espacio de búsqueda es grande.

In [109]:
#crearmos diccionarios para los distintos modelos

# regresión Logística
log_reg_param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100],  # regularización
    'classifier__solver': ['liblinear', 'saga']  # solvers disponibles
}

# K-Nearest Neighbors (KNN)
knn_param_grid = {
    'classifier__n_neighbors': [1, 3, 5, 7, 9],  # número de vecinos
    'classifier__weights': ['uniform', 'distance'],  # método de ponderación
    'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  # algoritmos de búsqueda de vecinos
}

# arbol de Decisión
tree_param_grid = {
    'classifier__max_depth': [3, 5, 10, None],  # profundidad máxima
    'classifier__min_samples_split': [2, 5, 10],  # mínimo número de muestras para dividir un nodo
    'classifier__min_samples_leaf': [1, 2, 4],  # mínimo número de muestras en una hoja
    'classifier__criterion': ['gini', 'entropy']  # criterio de división
}

# modelos a optimizar con GridSearchCV
models = {
    "Regresión Logística": LogisticRegression(), # creamos una instancia de Regresión Logística
    "K-Nearest Neighbors (KNN)": KNeighborsClassifier(), # creamos una instancia de K-Nearest Neighbors (KNN)
    "Árbol de Decisión": DecisionTreeClassifier(random_state=42)  # creamos una instancia de Árbol de Decisión con semilla aleatoria 42
}

# diccionario para guardar los resultados de GridSearchCV
grid_search_results = {}

In [110]:
# creamos el pipeline con preprocesamiento y clasificación
pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('genre_writer', OneHotEncoder(handle_unknown='ignore'), ['Genre', 'Writer']), # realizamos la codificación One-Hot para 'Genre' y 'Writer'
            ('scaler', StandardScaler(), ['Rating']) # escalamos la columna 'Rating' para normalizarla
        ],
        remainder='passthrough' # dejamos las demás columnas sin cambios
    )),
    ('classifier', LogisticRegression())  # asignamos un modelo de regresión logística por defecto
])

# dividimos el conjunto de datos en entrenamiento y prueba
X = df[['Genre', 'Writer', 'Likes', 'Subscribers', 'Rating']]  # seleccionamos las características del modelo
y = df['high_rating']  # definimos la variable objetivo

# dividimos los datos en 70% entrenamiento y 30% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [111]:
# ejecutamos GridSearchCV para cada modelo
for model_name, model in models.items():
    if model_name == "Regresión Logística": # si el modelo es Regresión Logística
        param_grid = log_reg_param_grid # usamos los parámetros específicos para Regresión Logística
    elif model_name == "K-Nearest Neighbors (KNN)": # si el modelo es KNN
        param_grid = knn_param_grid # usamos los parámetros específicos para KNN
    else:  # sino
        param_grid = tree_param_grid # usamos los parámetros específicos para el Árbol de Decisión
    
    # reemplazamos el clasificador en el pipeline
    pipeline.set_params(classifier=model) # actualizamos el clasificador en el pipeline con el modelo actual
        
    # creamos GridSearchCV
    grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    
    # ejecutamos GridSearchCV
    grid_search.fit(X_train, y_train) 
    
    # guardar los resultados
    grid_search_results[model_name] = { # almacenamos los resultados de la búsqueda para cada modelo
        'Best Score': grid_search.best_score_, # puntuación de la mejor combinación de parámetros
        'Best Params': grid_search.best_params_, # los mejores parámetros encontrados
        'All Results': grid_search.cv_results_ # todos los resultados de la búsqueda
    }

/home/cvelazquez/anaconda3/envs/ds_mitic/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/cvelazquez/anaconda3/envs/ds_mitic/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/cvelazquez/anaconda3/envs/ds_mitic/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/cvelazquez/anaconda3/envs/ds_mitic/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/cvelazquez/anaconda3/envs/ds_mitic/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

In [112]:
# mostramos los resultados de GridSearchCV para cada modelo
for model_name, result in grid_search_results.items():
    print("=" * 60)
    print(f"Resultados de GridSearchCV para {model_name}:")
    print("-" * 60)
    print(f"Mejor Precisión: {result['Best Score']:.4f}")
    print(f"Mejores Parámetros: {result['Best Params']}\n")

Resultados de GridSearchCV para Regresión Logística:
------------------------------------------------------------
Mejor Precisión: 0.5100
Mejores Parámetros: {'classifier__C': 0.01, 'classifier__solver': 'liblinear'}

Resultados de GridSearchCV para K-Nearest Neighbors (KNN):
------------------------------------------------------------
Mejor Precisión: 0.6358
Mejores Parámetros: {'classifier__algorithm': 'auto', 'classifier__n_neighbors': 9, 'classifier__weights': 'uniform'}

Resultados de GridSearchCV para Árbol de Decisión:
------------------------------------------------------------
Mejor Precisión: 1.0000
Mejores Parámetros: {'classifier__criterion': 'gini', 'classifier__max_depth': 3, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2}



### 3. Optuna:
* Implementar Optuna para una optimización avanzada de los hiperparámetros, aprovechando técnicas como la optimización bayesiana y el pruning.

In [113]:
def objective_log_reg(trial):
    # hiperparámetros de la Regresión Logística a optimizar
    C = trial.suggest_loguniform('C', 1e-5, 1e2) # sugerimos un valor para el parámetro C usando una distribución logarítmica
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga']) # sugerimos un valor para el parámetro solver entre 'liblinear' y 'saga'
    
    # creamos el modelo
    model = Pipeline([ # definimos el pipeline que incluye preprocesamiento y el clasificador
        ('preprocessor', preprocessor), # preprocesamos los datos con el pipeline predefinido
        ('classifier', LogisticRegression(C=C, solver=solver, random_state=42)) #  aplicamos el clasificador de Regresión Logística con los hiperparámetros sugeridos
    ])
    
    # entrenamiento y evaluación
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    # retorno de la precisión
    return accuracy_score(y_test, y_pred)

In [114]:
def objective_knn(trial):
    # hiperparámetros de KNN a optimizar
    n_neighbors = trial.suggest_int('n_neighbors', 1, 10) # sugerimos el número de vecinos entre 1 y 10
    weights = trial.suggest_categorical('weights', ['uniform', 'distance']) # sugerimos el tipo de pesos ('uniform' o 'distance')
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute']) # sugerimos el algoritmo de búsqueda para KNN
    # creación del modelo
    model = Pipeline([
        ('preprocessor', preprocessor),  # aplicamos el preprocesamiento definido previamente
        ('classifier', KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, algorithm=algorithm)) # aplicamos el clasificador KNN con los hiperparámetros sugeridos
    ])
    # entrenamiento y evaluación
    model.fit(X_train, y_train) # entrenamos el modelo con los datos de entrenamiento
    y_pred = model.predict(X_test)  # realizamos las predicciones sobre el conjunto de prueba
    # retornar la precisión
    return accuracy_score(y_test, y_pred)

In [115]:
def objective_tree(trial):
    # hiperparámetros del Árbol de Decisión a optimizar
    max_depth = trial.suggest_int('max_depth', 3, 10) # sugerimos la profundidad máxima del árbol entre 3 y 10
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10) # sugerimos el número mínimo de muestras para dividir un nodo entre 2 y 10
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4) # sugerimos el número mínimo de muestras por hoja entre 1 y 4
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy']) # sugerimos el criterio de división ('gini' o 'entropy')
    # creación del modelo
    model = Pipeline([
        ('preprocessor', preprocessor), # aplicamos el preprocesamiento definido previamente
        ('classifier', DecisionTreeClassifier(  # configuramos el clasificador Árbol de Decisión con los parámetros sugeridos
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            criterion=criterion,
            random_state=42 # fijamos la semilla para la aleatoriedad
        ))
    ])
    # entrenamiento y evaluación
    model.fit(X_train, y_train)
    # realizamos las predicciones sobre el conjunto de prueba
    y_pred = model.predict(X_test)
    # retornar la precisión
    return accuracy_score(y_test, y_pred)

In [116]:
# creamos un estudio de optimización utilizando Optuna
# direction='maximize' indica que estamos buscando maximizar la métrica (en este caso, la precisión)
# study_name asigna un nombre identificador al estudio para facilitar su seguimiento
log_reg_study = optuna.create_study(direction='maximize', study_name='Logistic Regression Optimization')
# ejecutamos la optimización del estudio, pasando la función objetivo 'objective_log_reg'
# n_trials=50 indica que queremos realizar 50 intentos de optimización, probando diferentes combinaciones de hiperparámetros
log_reg_study.optimize(objective_log_reg, n_trials=50)

[I 2024-11-20 15:53:53,689] A new study created in memory with name: Logistic Regression Optimization
/tmp/ipykernel_6557/2367070094.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e2) # sugerimos un valor para el parámetro C usando una distribución logarítmica
[I 2024-11-20 15:53:53,727] Trial 0 finished with value: 0.9824561403508771 and parameters: {'C': 6.626860080450494, 'solver': 'liblinear'}. Best is trial 0 with value: 0.9824561403508771.
/tmp/ipykernel_6557/2367070094.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e2) # sugerimos un valor para el parámetro C usando una di

In [117]:
# creamos un estudio de optimización utilizando Optuna para el modelo KNN
# direction='maximize' indica que estamos buscando maximizar la métrica (en este caso, la precisión)
# study_name asigna un nombre identificador al estudio para facilitar su seguimiento
knn_study = optuna.create_study(direction='maximize', study_name='KNN Optimization')
# ejecutamos la optimización del estudio, pasando la función objetivo 'objective_knn'
# n_trials=50 indica que queremos realizar 50 intentos de optimización, probando diferentes combinaciones de hiperparámetros
knn_study.optimize(objective_knn, n_trials=50)


[I 2024-11-20 15:54:29,908] A new study created in memory with name: KNN Optimization
/home/cvelazquez/anaconda3/envs/ds_mitic/lib/python3.10/site-packages/sklearn/neighbors/_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
[I 2024-11-20 15:54:29,946] Trial 0 finished with value: 0.9122807017543859 and parameters: {'n_neighbors': 4, 'weights': 'uniform', 'algorithm': 'ball_tree'}. Best is trial 0 with value: 0.9122807017543859.
/home/cvelazquez/anaconda3/envs/ds_mitic/lib/python3.10/site-packages/sklearn/neighbors/_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
[I 2024-11-20 15:54:29,981] Trial 1 finished with value: 0.8596491228070176 and parameters: {'n_neighbors': 7, 'weights': 'distance', 'algorithm': 'kd_tree'}. Best is trial 0 with value: 0.9122807017543859.
/home/cvelazquez/an

In [118]:
# creamos un estudio de optimización utilizando Optuna para el modelo de Árbol de Decisión
# direction='maximize' indica que estamos buscando maximizar la métrica (en este caso, la precisión)
# study_name asigna un nombre identificador al estudio para facilitar su seguimiento
tree_study = optuna.create_study(direction='maximize', study_name='Decision Tree Optimization')
# ejecutamos la optimización del estudio, pasando la función objetivo 'objective_tree'
# n_trials=50 indica que queremos realizar 50 intentos de optimización, probando diferentes combinaciones de hiperparámetros
tree_study.optimize(objective_tree, n_trials=50)


[I 2024-11-20 15:55:00,533] A new study created in memory with name: Decision Tree Optimization
[I 2024-11-20 15:55:00,567] Trial 0 finished with value: 1.0 and parameters: {'max_depth': 3, 'min_samples_split': 10, 'min_samples_leaf': 2, 'criterion': 'entropy'}. Best is trial 0 with value: 1.0.
[I 2024-11-20 15:55:00,611] Trial 1 finished with value: 1.0 and parameters: {'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 1, 'criterion': 'gini'}. Best is trial 0 with value: 1.0.
[I 2024-11-20 15:55:00,650] Trial 2 finished with value: 1.0 and parameters: {'max_depth': 5, 'min_samples_split': 4, 'min_samples_leaf': 4, 'criterion': 'gini'}. Best is trial 0 with value: 1.0.
[I 2024-11-20 15:55:00,717] Trial 3 finished with value: 1.0 and parameters: {'max_depth': 7, 'min_samples_split': 7, 'min_samples_leaf': 4, 'criterion': 'gini'}. Best is trial 0 with value: 1.0.
[I 2024-11-20 15:55:00,790] Trial 4 finished with value: 1.0 and parameters: {'max_depth': 10, 'min_samples_split':

In [119]:
# mostramos los resultados de las purebas
print(f'Mejor prueba para Logistic Regression: {log_reg_study.best_trial.params}')
print(f'Mejor prueba para KNN: {knn_study.best_trial.params}')
print(f'Mejor prueba para Decision Tree: {tree_study.best_trial.params}')

Mejor prueba para Logistic Regression: {'C': 77.82162145535581, 'solver': 'liblinear'}
Mejor prueba para KNN: {'n_neighbors': 4, 'weights': 'uniform', 'algorithm': 'ball_tree'}
Mejor prueba para Decision Tree: {'max_depth': 3, 'min_samples_split': 10, 'min_samples_leaf': 2, 'criterion': 'entropy'}


In [121]:
# regresión Logística
best_log_reg_params = log_reg_study.best_trial.params
# obtenemos los mejores parámetros encontrados por Optuna para la Regresión Logística

log_reg_best_model = Pipeline([
    ('preprocessor', preprocessor),  # aplicamos el preprocesamiento previamente definido
    ('classifier', LogisticRegression(C=best_log_reg_params['C'], solver=best_log_reg_params['solver'], random_state=42))
    # configuramos el modelo de Regresión Logística con los mejores parámetros obtenidos
])
log_reg_best_model.fit(X_train, y_train)
# entrenamos el modelo con los datos de entrenamiento

log_reg_accuracy = accuracy_score(y_test, log_reg_best_model.predict(X_test))
# calculamos la precisión del modelo sobre el conjunto de prueba

# KNN
best_knn_params = knn_study.best_trial.params
# obtenemos los mejores parámetros encontrados por Optuna para el modelo KNN

knn_best_model = Pipeline([
    ('preprocessor', preprocessor),  # aplicamos el preprocesamiento previamente definido
    ('classifier', KNeighborsClassifier(n_neighbors=best_knn_params['n_neighbors'], 
                                        weights=best_knn_params['weights'], 
                                        algorithm=best_knn_params['algorithm']))
    # configuramos el modelo KNN con los mejores parámetros obtenidos
])
knn_best_model.fit(X_train, y_train) # entrenamos el modelo KNN con los datos de entrenamiento

knn_accuracy = accuracy_score(y_test, knn_best_model.predict(X_test))
# calculamos la precisión del modelo KNN sobre el conjunto de prueba

# Árbol de Decisión
best_tree_params = tree_study.best_trial.params
# obtenemos los mejores parámetros encontrados por Optuna para el Árbol de Decisión
tree_best_model = Pipeline([
    ('preprocessor', preprocessor),  # aplicamos el preprocesamiento previamente definido
    ('classifier', DecisionTreeClassifier(
        max_depth=best_tree_params['max_depth'],
        min_samples_split=best_tree_params['min_samples_split'],
        min_samples_leaf=best_tree_params['min_samples_leaf'],
        criterion=best_tree_params['criterion'],
        random_state=42
    ))
    # configuramos el modelo de Árbol de Decisión con los mejores parámetros obtenidos
])
tree_best_model.fit(X_train, y_train)
# entrenamos el modelo Árbol de Decisión con los datos de entrenamiento

tree_accuracy = accuracy_score(y_test, tree_best_model.predict(X_test))
# calculamos la precisión del modelo Árbol de Decisión sobre el conjunto de prueba


/home/cvelazquez/anaconda3/envs/ds_mitic/lib/python3.10/site-packages/sklearn/neighbors/_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


In [122]:
# Resultados finales
print(f'Logistic Regression Accuracy: {log_reg_accuracy:.4f}')
print(f'KNN Accuracy: {knn_accuracy:.4f}')
print(f'Decision Tree Accuracy: {tree_accuracy:.4f}')

Logistic Regression Accuracy: 0.9883
KNN Accuracy: 0.9123
Decision Tree Accuracy: 1.0000


### 4. Evaluación de Modelos Optimizados:
* Entrenar el modelo con los mejores hiperparámetros encontrados y evaluar su rendimiento en el conjunto de prueba.
* Comparar el rendimiento del modelo optimizado con el modelo inicial.

In [123]:
# obtener los mejores parámetros de Logistic Regression
best_log_reg_params = log_reg_study.best_trial.params

# creamos el modelo con los mejores parámetros
log_reg_best_model = Pipeline([
    ('preprocessor', preprocessor),  # aplicamos el preprocesamiento previamente definido
    ('classifier', LogisticRegression(
        C=best_log_reg_params['C'],  # asignamos el mejor valor para C
        solver=best_log_reg_params['solver'],  # asignamos el mejor valor para solver
        random_state=42  # aseguramos reproducibilidad del modelo
    ))
])

# Entrenar el modelo
log_reg_best_model.fit(X_train, y_train) 

# Evaluación del rendimiento
log_reg_y_pred = log_reg_best_model.predict(X_test) # realizamos las predicciones sobre el conjunto de prueba
log_reg_accuracy = accuracy_score(y_test, log_reg_y_pred) # calculamos la precisión del modelo (proporción de predicciones correctas)
log_reg_precision = precision_score(y_test, log_reg_y_pred) # calculamos la precisión (proporción de predicciones positivas correctas)
log_reg_recall = recall_score(y_test, log_reg_y_pred) # calculamos el recall (proporción de verdaderos positivos identificados correctamente)
log_reg_f1 = f1_score(y_test, log_reg_y_pred) # calculamos el F1-score (media armónica de precisión y recall)
log_reg_roc_auc = roc_auc_score(y_test, log_reg_y_pred) # calculamos el área bajo la curva ROC (medida de la capacidad del modelo para diferenciar entre clases)

# mostramos los resultados de la evaluación
print("Logistic Regression Performance:")
print(f"Accuracy: {log_reg_accuracy:.4f}") # mostramos la precisión del modelo con 4 decimales
print(f"Precision: {log_reg_precision:.4f}") # mostramos la precisión del modelo con 4 decimales
print(f"Recall: {log_reg_recall:.4f}") # mostramos el recall del modelo con 4 decimales
print(f"F1-Score: {log_reg_f1:.4f}")# mostramos el F1-score del modelo con 4 decimales
print(f"ROC-AUC: {log_reg_roc_auc:.4f}") # mostramos el ROC-AUC del modelo con 4 decimales


Logistic Regression Performance:
Accuracy: 0.9883
Precision: 0.9750
Recall: 1.0000
F1-Score: 0.9873
ROC-AUC: 0.9892


In [124]:
# obtenemos los mejores parámetros de KNN
best_knn_params = knn_study.best_trial.params
# creamos el modelo con los mejores parámetros
knn_best_model = Pipeline([
    ('preprocessor', preprocessor),  # aplicamos el preprocesamiento previamente definido
    ('classifier', KNeighborsClassifier(
        n_neighbors=best_knn_params['n_neighbors'],  # asignamos el mejor valor para n_neighbors
        weights=best_knn_params['weights'],  # asignamos el mejor valor para weights
        algorithm=best_knn_params['algorithm']  # asignamos el mejor valor para algorithm
    ))
])
# entrenamos el modelo
knn_best_model.fit(X_train, y_train) 
knn_y_pred = knn_best_model.predict(X_test) # realizamos las predicciones sobre el conjunto de prueba
knn_accuracy = accuracy_score(y_test, knn_y_pred) # calculamos la precisión del modelo (proporción de predicciones correctas)
knn_precision = precision_score(y_test, knn_y_pred) # calculamos la precisión (proporción de predicciones positivas correctas)
knn_recall = recall_score(y_test, knn_y_pred) # calculamos el recall (proporción de verdaderos positivos identificados correctamente)
knn_f1 = f1_score(y_test, knn_y_pred) # calculamos el F1-score (media armónica de precisión y recall)
knn_roc_auc = roc_auc_score(y_test, knn_y_pred) # calculamos el área bajo la curva ROC (medida de la capacidad del modelo para diferenciar entre clases)

# mostramos los resultados de la evaluación
print("\nK-Nearest Neighbors Performance:")
print(f"Accuracy: {knn_accuracy:.4f}") # mostramos la precisión del modelo con 4 decimales
print(f"Precision: {knn_precision:.4f}") # mostramos la precisión del modelo con 4 decimales
print(f"Recall: {knn_recall:.4f}") # mostramos el recall del modelo con 4 decimales
print(f"F1-Score: {knn_f1:.4f}") # mostramos el F1-score del modelo con 4 decimales
print(f"ROC-AUC: {knn_roc_auc:.4f}") # mostramos el ROC-AUC del modelo con 4 decimales



K-Nearest Neighbors Performance:
Accuracy: 0.9123
Precision: 0.8987
Recall: 0.9103
F1-Score: 0.9045
ROC-AUC: 0.9121


/home/cvelazquez/anaconda3/envs/ds_mitic/lib/python3.10/site-packages/sklearn/neighbors/_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


In [125]:
# Obtener los mejores parámetros de Decision Tree
best_tree_params = tree_study.best_trial.params
# Crear el modelo con los mejores parámetros
tree_best_model = Pipeline([
    ('preprocessor', preprocessor),  # aplicamos el preprocesamiento previamente definido
    ('classifier', DecisionTreeClassifier(
        max_depth=best_tree_params['max_depth'],  # asignamos el mejor valor para max_depth
        min_samples_split=best_tree_params['min_samples_split'],  # asignamos el mejor valor para min_samples_split
        min_samples_leaf=best_tree_params['min_samples_leaf'],  # asignamos el mejor valor para min_samples_leaf
        criterion=best_tree_params['criterion'],  # asignamos el mejor valor para criterion
        random_state=42  # aseguramos la reproducibilidad del modelo con un valor fijo para random_state
    ))
])

# entrenamos el modelo
tree_best_model.fit(X_train, y_train)

tree_y_pred = tree_best_model.predict(X_test) # realizamos las predicciones sobre el conjunto de prueba
tree_accuracy = accuracy_score(y_test, tree_y_pred) # calculamos la precisión del modelo (proporción de predicciones correctas)
tree_precision = precision_score(y_test, tree_y_pred) # calculamos la precisión (proporción de predicciones positivas correctas)
tree_recall = recall_score(y_test, tree_y_pred) # calculamos el recall (proporción de verdaderos positivos identificados correctamente)
tree_f1 = f1_score(y_test, tree_y_pred) # calculamos el F1-score (media armónica de precisión y recall)
tree_roc_auc = roc_auc_score(y_test, tree_y_pred) # calculamos el área bajo la curva ROC (medida de la capacidad del modelo para diferenciar entre clases)

# mostramos los resultados de la evaluación
print("\nDecision Tree Performance:")
print(f"Accuracy: {tree_accuracy:.4f}") # mostramos la precisión del modelo con 4 decimales
print(f"Precision: {tree_precision:.4f}") # mostramos la precisión del modelo con 4 decimales
print(f"Recall: {tree_recall:.4f}") # mostramos el recall del modelo con 4 decimales
print(f"F1-Score: {tree_f1:.4f}") # mostramos el F1-score del modelo con 4 decimales
print(f"ROC-AUC: {tree_roc_auc:.4f}") # mostramos el ROC-AUC del modelo con 4 decimales



Decision Tree Performance:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
ROC-AUC: 1.0000
